In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise04AI import Exercise04AI
from Exercise06AI import Exercise06AI

# Aufgabe 07Lite: Minimax mit Alpha-Beta-Pruning und Progressive Deepening OHNE Memoisierung

Dieses Notebook ist eine Kopie der `Exercise07AI` ohne Memoisierung. Durch diese Modifizierung sinkt der verwendete Arbeitsspeicher erheblich und die Berechnungsdauer erhöht sich leicht.

In [ ]:
import chess_custom as chess
from typing import Any

class Exercise07LiteAI(Exercise06AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning and memoization"""
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        self.local_cache: dict[tuple, int] = {}

## Entfernung der Memoisierung

Diese Klasse ist bis auf das Entfernen der Memoisierung identisch zur Klasse `Exercise07AI`.

In [ ]:
from typing import Any
import heapq

class Exercise07LiteAI(Exercise07LiteAI): # type: ignore
    
    def minimax(self, board: chess.Board, depth: int, current_evaluation: int,
                alpha: int = -Exercise07LiteAI.LIMIT, beta: int = Exercise07LiteAI.LIMIT) -> tuple[int, chess.Move]:
        """Searches the best value with given depth using minimax algorithm"""
        best_move = None
        
        if (is_checkmate := board.is_checkmate()) and not board.turn:
            # White has won the game
            evaluation = self.LIMIT - (self.DEPTH - depth)
            return evaluation, None
        elif is_checkmate and board.turn:
            # Black has won the game
            evaluation = -self.LIMIT + (self.DEPTH - depth)
            return evaluation, None
        elif board.is_insufficient_material() or not board.legal_moves or board.is_fifty_moves() or board.is_repetition(5):
            # Game is a draw
            return 0, None

        # Recursion abort case
        if depth == 0:
            return current_evaluation, None

        # White to play (positive numbers are good)
        moves: list[tuple[int, int, chess.Move]] = []
        if board.turn:
            for i, move in enumerate(board.legal_moves):
                old_eval = self.local_cache.get((depth - 2, move), i)
                heapq.heappush(moves, (-old_eval, i, move))
            maxEvaluation = alpha
            while moves:
                move = heapq.heappop(moves)[2]
                board.push(move)
                evaluation, _ = self.minimax(board, depth - 1, self.evaluate(board, current_evaluation), maxEvaluation, beta)
                self.local_cache[(depth - 1, move)] = evaluation
                board.pop()
                if evaluation >= beta:
                    return evaluation, move
                if depth == self.DEPTH and evaluation > maxEvaluation:
                    best_move = move
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation, best_move

        # Black to play (negative numbers are good)
        else:
            for i, move in enumerate(board.legal_moves):
                old_eval = self.local_cache.get((depth - 2, move), i)
                heapq.heappush(moves, (old_eval, i, move))
            minEvaluation = beta
            while moves:
                move = heapq.heappop(moves)[2]
                board.push(move)
                evaluation, _ = self.minimax(board, depth - 1, self.evaluate(board, current_evaluation), alpha, minEvaluation)
                self.local_cache[(depth - 1, move)] = evaluation
                board.pop()
                if evaluation <= alpha:
                    return evaluation, move
                if depth == self.DEPTH and evaluation < minEvaluation:
                    best_move = move
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation, best_move

In [ ]:
class Exercise07LiteAI(Exercise07LiteAI): # type: ignore
    
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move | None:
        """Gets the best next move"""
        self.last_evaluation: int | None # type annotation for mypy
        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        # Calculate current evaluation
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            current_evaluation = self.evaluate(board, self.last_evaluation)

        for depth in range(1, self.DEPTH + 1):
            # Call minimax and get best move
            _, best_move = self.minimax(board, depth, current_evaluation)
        
        # Reset local cache
        self.local_cache: dict[tuple, int] = {}
        
        # Debugging fail save
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = current_evaluation + self.incremental_evaluate(board, best_move)
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
#debug
from Exercise03AI import Exercise03AI

In [ ]:
#debug
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
%%time
DEPTH = 4
player3 = Exercise07LiteAI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player3.get_next_middle_game_move(board)
print(move)

In [ ]:
%%time
DEPTH = 4
player5 = Exercise05AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player5.get_next_middle_game_move(board)
print(move)

In [ ]:
%%time
DEPTH = 4
player6 = Exercise06AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player6.get_next_middle_game_move(board)
print(move)
print(player6.cache_hits)

In [ ]:
#debug
from IPython.display import clear_output, display
player5 = Exercise05AI("Testplayer", 3)
player3 = Exercise03AI("Testplayer", 3)
move = player3.random.choice(list(board.legal_moves))
board.push(move)
for _ in range(10):
    move_1, eval_1 = player3.get_next_middle_game_move(board)
    player3.last_evaluation = None
    print(move_1, eval_1, player3.last_evaluation)
    move_2, eval_2 = player5.get_next_middle_game_move(board)
    player5.last_evaluation = None
    print(move_2, eval_2, player5.last_evaluation)
    move = player3.random.choice(list(board.legal_moves))
    board.push(move)

In [ ]:
%%time
DEPTH = 3
board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
for _ in range(10):
    board.push(sorted(board.legal_moves, key=lambda move: move.uci())[0])
    move = player.get_next_middle_game_move(board)
    print(move)

In [ ]:
#debug
from os.path import join

with open(join("..", "games", "2022-01-15_20-18-57-907777.pgn")) as pgn:
    first_game = chess.pgn.read_game(pgn)

# Iterate through all moves and play them on a board.
board = first_game.board()
for move in first_game.mainline_moves():
    board.push(move)
for i in range(220):
    board.pop()

In [ ]:
%%time
DEPTH = 2
#board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.get_next_middle_game_move(board)
print(move)
player.cache_hits

In [ ]:
#debug
board = chess.Board("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise04AI("Testplayer", 3)
print(player.evaluate(board))
board